# Подготовка данных

In [25]:
%matplotlib inline
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import sklearn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [26]:
# df = pd.read_csv(r"InternationalBifurca_DATA_2023-10-30_0629.csv", sep=',')
df = pd.read_csv(r"InternationalBifurca_DATA_2025-04-20_0932.csv", sep=',')

/var/folders/2x/tx689lxn54ngmttr2b89rvg00000gn/T/ipykernel_8308/3738330455.py:2: DtypeWarning: Columns (264) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"InternationalBifurca_DATA_2025-04-20_0932.csv", sep=',')


In [28]:
df = df.dropna(subset=['sex'])
df

,record_id,identifier,date,adhoc_pci,sex,age,weight,height,clinical_presentation,time_from_mi_symptoms_onse,...,time_to_death_f5,time_to_acs_f5,time_to_stroke_f5,time_to_pci_f5,time_to_cabg_f5,hospitalization_f5,bleeding_f5,angio_follow_f5,restenosis_f5,side_branch_restenosis_5
0,MNRI0001,MNRI-2018-0001,2018-02-01,1.0,2.0,77.0,84.0,165.0,5.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MNRI0002,MNRI-2018-0002,2018-01-24,0.0,1.0,68.0,81.0,171.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MNRI0003,MNRI-2018-0003,2018-01-24,0.0,1.0,62.0,74.0,180.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MNRI0004,MNRI-2018-0004,2018-01-30,1.0,1.0,67.0,84.0,167.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MNRI0005,MNRI-2018-0005,2018-01-30,0.0,1.0,57.0,103.0,174.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139,TRCH0026,TRCH-2019-0026,2019-03-11,1.0,1.0,67.0,90.0,174.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2140,TRCH0027,TRCH-2019-0027,2019-03-18,1.0,1.0,69.0,60.0,174.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2141,TRCH0028,TRCH-2019-0028,2019-03-19,0.0,2.0,81.0,50.0,160.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2142,TRCH0029,TRCH-2019-0029,2019-03-28,1.0,1.0,86.0,74.0,170.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
anyInf = df[df == np.inf].sum()
anyInf[anyInf != 0]

stent_distal_vessel_size    inf
sb_stent_sb_diametr         inf
dtype: object

In [30]:
patient_info_cols = ['record_id', 'date', 'sex', 'age', 'adhoc_pci', 'weight', 'height', 
                     'clinical_presentation', 'time_from_mi_symptoms_onse', 
                     'ccs_class', 'diabet', 'insulin_diabetes', 'hypertension', 'smoking', 
                     'dyslipidemia', 'anemia', 'atrial_fibrilation', 'oac_use', 'valvular_disease', 
                     'valvular_disease_was_previ', 'if_yes_what_type___1', 'if_yes_what_type___2', 
                     'if_yes_what_type___3', 'if_yes_what_type___4', 'if_yes_what_type___5', 
                     'if_yes_what_type___6', 'if_yes_what_type___7', 'ef', 'creatinine', 'ckd', 
                     'mi_history', 'cerebrovascular_disease', 'previously_treated_cerebro', 'previous_stroke_tia', 
                     'peripheral_artery_disease', 'previously_treated_periphe', 'copd', 'history_of_cancer', 
                     'previous_pci', 'previous_cabg']

intervention_cols = ['single_vessel', 'trifurcation', 'bifurcation_location', 
                  'lesion_ivolves', 'angle', 'calcium', 'trombosis', 
                  'total_trobotic_occlusion', 'restenosis_reocclusion', 'cto_bifurc', 
                  'medina_proximal', 'medina_distal', 'medina_side', 'mb_length_proximal', 
                  'sb_length', 'proximal_diametr', 'distal_diametr', 'side_diametr', 'stenosis_proximal', 
                  'stenosis_distal', 'timi_flow_main_branch', 'side_stenosis', 'timi_flow_side_branch', 
                  'major_lm', 'major_non_lm', 'minor_criteria', 'main_branch_rvd', 'def', 'def_2']

operation_cols = ['side_protection', 'main_predilatation', 'side_predilat', 
                  'stent_was_implated_from_lm', 'stent_number', 'stent_number_bif', 'stent_technique', 
                  'first_stent_impanted', 'provisional_2_stent_techni', 'stent_direction', 'defered_stenting', 
                  'stent_diameter', 'stent_length', 'stent_type___1', 'stent_type___2', 'stent_type___3', 
                  'stent_type___4', 'stent_type___5', 'stent_type___6', 'stent_type___7', 'stent_type___9', 
                  'stent_type___8', 'dstent2', 'stent_length2', 'stent_distal_vessel_size', 
                  'sb_stent_side_branch_diametr', 'sb_stent_sb_diametr', 'twostent_technique', 
                  'sb_dilatation', 'stent_postdilatation', 'proximal_optimization', 'pot', 
                  'pot_balloon_diametr', 'kissing_post', 'modified_kis', 'several_kissing']

new_cols = ['adverse_event_followup_f2_v2', 'angio_follow_f5', 'antiplatalet_drug_was_chan',
            'attempt_to_dilate_stenting', 'ballooon_size_for_postdila', 'complete_revascularisation',
            'currently_on_dialysis', 'followup_1_year_do_not_complete_if_2nd_bifurcation_complete',
            'identifier', 'ishemia_test___1', 'ishemia_test___2', 'ishemia_test___3', 'ishemia_test___4',
            'kissing_post_2stent___1', 'kissing_post_2stent___2', 'left_main_stent_direction',
            'main_branch_calcification', 'mb_stenosis_f2', 'medina_side_branch_2', 'myocardial_ischemia',
            'myocardial_ishemia_was_det', 'number_of_kissing', 'number_of_kissing_2',
            'other_lesions_in_main_bran', 'other_lesions_in_side_brach',
            'patient_information_do_not_complete_if_2nd_bifurca_complete', 'pot_2', 'pot_balloon_diametr_2',
            'pot_balloon_length', 'pot_balloon_length_2', 'pressure2', 'reson_for_change_stopped___1',
            'reson_for_change_stopped___2', 'reson_for_change_stopped___3', 'restenosis_f5', 'sb_length_2',
            'sb_stenosis_f2', 'side_branch_calcification_2', 'side_branch_restenosis',
            'side_branch_restenosis_3', 'side_branch_restenosis_5', 'stent_pressure', 'stent_type_2___1',
            'stent_type_2___2', 'stent_type_2___3', 'stent_type_2___4', 'stent_type_2___5', 'stent_type_2___6',
            'stent_type_2___7', 'stent_type_2___8', 'stent_type_2___9', 'thrombolysis', 'uncross_strategy___1',
            'uncross_strategy___2', 'uncross_strategy___3', 'uncross_strategy___4', 'uncross_strategy___5',
            'uncross_strategy___6', 'uncross_strategy___7', 'uncross_strategy___8', 'uncross_strategy___9']

In [31]:
non_vascular_deaths = ['MNRI1054', 'MNRI1191', 'MNRI1351', 'MNRI1352', 'MNRI1473', 'MNRI1670', 'MNRI0637', 'MNRI0656', 'MNRI0751', 'MNRI0758',
                      'MNRI0805', 'MNRI0818', 'MNRI1054', 'MNRI0087', 'MNRI1191', 'MNRI0108', 'MNRI0307', 'MNRI0215', 'MNRI0322', 'MNRI0293',
                      'MNRI0156', 'MNRI0215', 'MNRI0488', 'MNRI0612', 'MNRI0708', 'MNRI0767', 'MNRI0772', 'MNRI0786', 'MNRI1105', 'MNRI1186',
                      'MNRI1462', 'MNRI1633']

In [32]:
df = df[~df['record_id'].isin(non_vascular_deaths)]

In [33]:
df

,record_id,identifier,date,adhoc_pci,sex,age,weight,height,clinical_presentation,time_from_mi_symptoms_onse,...,time_to_death_f5,time_to_acs_f5,time_to_stroke_f5,time_to_pci_f5,time_to_cabg_f5,hospitalization_f5,bleeding_f5,angio_follow_f5,restenosis_f5,side_branch_restenosis_5
0,MNRI0001,MNRI-2018-0001,2018-02-01,1.0,2.0,77.0,84.0,165.0,5.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MNRI0002,MNRI-2018-0002,2018-01-24,0.0,1.0,68.0,81.0,171.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MNRI0003,MNRI-2018-0003,2018-01-24,0.0,1.0,62.0,74.0,180.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MNRI0004,MNRI-2018-0004,2018-01-30,1.0,1.0,67.0,84.0,167.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MNRI0005,MNRI-2018-0005,2018-01-30,0.0,1.0,57.0,103.0,174.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139,TRCH0026,TRCH-2019-0026,2019-03-11,1.0,1.0,67.0,90.0,174.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2140,TRCH0027,TRCH-2019-0027,2019-03-18,1.0,1.0,69.0,60.0,174.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2141,TRCH0028,TRCH-2019-0028,2019-03-19,0.0,2.0,81.0,50.0,160.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2142,TRCH0029,TRCH-2019-0029,2019-03-28,1.0,1.0,86.0,74.0,170.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
patient_present_cols = patient_info_cols + intervention_cols + operation_cols + new_cols
patient_present_df = pd.DataFrame({col_name: df[col_name] for col_name in patient_present_cols})

In [35]:
patient_present_df

,record_id,date,sex,age,adhoc_pci,weight,height,clinical_presentation,time_from_mi_symptoms_onse,ccs_class,...,thrombolysis,uncross_strategy___1,uncross_strategy___2,uncross_strategy___3,uncross_strategy___4,uncross_strategy___5,uncross_strategy___6,uncross_strategy___7,uncross_strategy___8,uncross_strategy___9
0,MNRI0001,2018-02-01,2.0,77.0,1.0,84.0,165.0,5.0,4.0,NaN,...,0.0,0,0,0,0,0,0,0,0,0
1,MNRI0002,2018-01-24,1.0,68.0,0.0,81.0,171.0,1.0,NaN,1.0,...,NaN,0,0,0,0,0,0,0,0,0
2,MNRI0003,2018-01-24,1.0,62.0,0.0,74.0,180.0,4.0,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0
3,MNRI0004,2018-01-30,1.0,67.0,1.0,84.0,167.0,2.0,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0
4,MNRI0005,2018-01-30,1.0,57.0,0.0,103.0,174.0,1.0,NaN,2.0,...,NaN,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139,TRCH0026,2019-03-11,1.0,67.0,1.0,90.0,174.0,2.0,NaN,NaN,...,0.0,0,0,0,0,0,0,0,0,0
2140,TRCH0027,2019-03-18,1.0,69.0,1.0,60.0,174.0,3.0,1.0,NaN,...,1.0,0,0,0,0,0,0,0,0,0
2141,TRCH0028,2019-03-19,2.0,81.0,0.0,50.0,160.0,2.0,NaN,NaN,...,0.0,0,0,0,0,0,0,0,0,0
2142,TRCH0029,2019-03-28,1.0,86.0,1.0,74.0,170.0,3.0,1.0,NaN,...,0.0,0,0,0,0,0,0,0,0,0


In [36]:
columns_to_preserve = ['valvular_disease', 'previous_stroke_tia', 'twostent_technique']

def remove_columns_with_nan_threshold(df, threshold=250):
    nan_counts = df.isnull().sum()
    
    columns_to_drop = [col for col in nan_counts[nan_counts > threshold].index 
                      if col not in columns_to_preserve]
    
    print(columns_to_drop)
    
    df_dropped = df.drop(columns=columns_to_drop)
    
    return df_dropped

In [37]:
patient_present_df = remove_columns_with_nan_threshold(patient_present_df, 500)

['time_from_mi_symptoms_onse', 'ccs_class', 'insulin_diabetes', 'valvular_disease_was_previ', 'previously_treated_cerebro', 'previously_treated_periphe', 'lesion_ivolves', 'total_trobotic_occlusion', 'sb_length', 'timi_flow_main_branch', 'timi_flow_side_branch', 'stent_was_implated_from_lm', 'first_stent_impanted', 'provisional_2_stent_techni', 'dstent2', 'stent_length2', 'sb_stent_side_branch_diametr', 'proximal_optimization', 'pot', 'pot_balloon_diametr', 'several_kissing', 'adverse_event_followup_f2_v2', 'angio_follow_f5', 'antiplatalet_drug_was_chan', 'attempt_to_dilate_stenting', 'ballooon_size_for_postdila', 'complete_revascularisation', 'left_main_stent_direction', 'main_branch_calcification', 'mb_stenosis_f2', 'medina_side_branch_2', 'myocardial_ischemia', 'myocardial_ishemia_was_det', 'number_of_kissing', 'number_of_kissing_2', 'other_lesions_in_main_bran', 'other_lesions_in_side_brach', 'pot_2', 'pot_balloon_diametr_2', 'pot_balloon_length', 'pot_balloon_length_2', 'pressure2

In [38]:
# patient_present_df = patient_present_df.drop(['record_id', 'date'], axis = 1)
patient_present_df = patient_present_df.drop(['date'], axis = 1)
patient_present_df

,record_id,sex,age,adhoc_pci,weight,height,clinical_presentation,diabet,hypertension,smoking,...,stent_type_2___9,uncross_strategy___1,uncross_strategy___2,uncross_strategy___3,uncross_strategy___4,uncross_strategy___5,uncross_strategy___6,uncross_strategy___7,uncross_strategy___8,uncross_strategy___9
0,MNRI0001,2.0,77.0,1.0,84.0,165.0,5.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,MNRI0002,1.0,68.0,0.0,81.0,171.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,MNRI0003,1.0,62.0,0.0,74.0,180.0,4.0,0.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,MNRI0004,1.0,67.0,1.0,84.0,167.0,2.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,MNRI0005,1.0,57.0,0.0,103.0,174.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139,TRCH0026,1.0,67.0,1.0,90.0,174.0,2.0,2.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2140,TRCH0027,1.0,69.0,1.0,60.0,174.0,3.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2141,TRCH0028,2.0,81.0,0.0,50.0,160.0,2.0,2.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2142,TRCH0029,1.0,86.0,1.0,74.0,170.0,3.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
patient_present_df.replace(to_replace = [np.inf, -np.inf], value= None, inplace=True)

In [40]:
cols_to_drop = patient_present_df.columns[patient_present_df.nunique() <= 1]
patient_present_df = patient_present_df.drop(cols_to_drop, axis=1)
patient_present_df

,record_id,sex,age,adhoc_pci,weight,height,clinical_presentation,diabet,hypertension,smoking,...,stent_type_2___5,stent_type_2___6,stent_type_2___7,stent_type_2___8,uncross_strategy___1,uncross_strategy___2,uncross_strategy___3,uncross_strategy___4,uncross_strategy___8,uncross_strategy___9
0,MNRI0001,2.0,77.0,1.0,84.0,165.0,5.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,MNRI0002,1.0,68.0,0.0,81.0,171.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,MNRI0003,1.0,62.0,0.0,74.0,180.0,4.0,0.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,MNRI0004,1.0,67.0,1.0,84.0,167.0,2.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,MNRI0005,1.0,57.0,0.0,103.0,174.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139,TRCH0026,1.0,67.0,1.0,90.0,174.0,2.0,2.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2140,TRCH0027,1.0,69.0,1.0,60.0,174.0,3.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2141,TRCH0028,2.0,81.0,0.0,50.0,160.0,2.0,2.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2142,TRCH0029,1.0,86.0,1.0,74.0,170.0,3.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
numerical = ['age', 'weight', 'height', 'ef', 'creatinine', 'ckd', 'angle', 'mb_length_proximal', 
            'proximal_diametr', 'distal_diametr', 'side_diametr', 'stenosis_proximal', 
            'stenosis_distal', 'side_stenosis', 'minor_criteria', 'main_branch_rvd', 
            'stent_diameter', 'stent_length', 'stent_distal_vessel_size', 'sb_stent_sb_diametr',
            'ballooon_size_for_postdila', 
            'left_main_stent_direction',
            'mb_stenosis_f2',
            'myocardial_ischemia',
            'number_of_kissing_2',
            'pot_balloon_diametr_2',
            'pot_balloon_length',
            'pot_balloon_length_2',
            'pressure2',
            'sb_length_2',
            'sb_stenosis_f2',
            'stent_pressure']

categorical = ['sex', 'clinical_presentation', 'bifurcation_location', 'stent_number', 
              'stent_number_bif', 'stent_technique', 'stent_direction']

binary = ['diabet', 'adhoc_pci', 'hypertension', 'smoking', 'dyslipidemia', 'anemia', 
         'atrial_fibrilation', 'oac_use', 'if_yes_what_type___1', 'if_yes_what_type___2',
         'if_yes_what_type___3', 'if_yes_what_type___4', 'if_yes_what_type___6', 
         'mi_history', 'cerebrovascular_disease', 'peripheral_artery_disease', 'copd', 
         'history_of_cancer', 'previous_pci', 'previous_cabg', 'single_vessel', 'trifurcation',
         'calcium', 'trombosis', 'restenosis_reocclusion', 'cto_bifurc', 
         'medina_proximal', 'medina_distal', 'medina_side', 'major_lm', 'major_non_lm',
         'def', 'def_2', 'side_protection', 'main_predilatation', 'side_predilat',
         'defered_stenting', 'stent_type___1', 'stent_type___2', 'stent_type___3', 'stent_type___4',
         'stent_type___5', 'stent_type___6', 'stent_type___7', 'stent_type___9', 'stent_type___8', 
         'sb_dilatation', 'stent_postdilatation', 'kissing_post', 'modified_kis',
         'currently_on_dialysis',
         'ishemia_test___1',
         'ishemia_test___2',
         'ishemia_test___3',
         'kissing_post_2stent___1',
         'kissing_post_2stent___2',
         'reson_for_change_stopped___1',
         'reson_for_change_stopped___2',
         'reson_for_change_stopped___3',
         'stent_type_2___1',
         'stent_type_2___3',
         'stent_type_2___4',
         'stent_type_2___5',
         'stent_type_2___6',
         'stent_type_2___7',
         'stent_type_2___8',
         'uncross_strategy___1',
         'uncross_strategy___2',
         'uncross_strategy___3',
         'uncross_strategy___4',
         'uncross_strategy___8',
         'uncross_strategy___9']

In [42]:
without_second_bif = df
adverse_events = without_second_bif['event_type_followup_f2___1'] \
| without_second_bif['event_type_followup_f2___2'] \
| without_second_bif['event_type_followup_f2_v2___1'] \
| without_second_bif['event_type_followup_f2_v2___2']
print(sum(adverse_events))
print(len(adverse_events))

158
2044


In [43]:
combined = zip(without_second_bif['event_type_followup_f2___1'],  
               without_second_bif['event_type_followup_f2___2'] * 2,
               without_second_bif['event_type_followup_f2_v2___1'] * 3, 
               without_second_bif['event_type_followup_f2_v2___2'] * 4)

combined_adverse_events = np.array([max(t) for t in combined])

In [44]:
unique, counts = np.unique(combined_adverse_events, return_counts=True)
value_counts = dict(zip(unique, counts))
print(value_counts)

{0: 1886, 1: 53, 2: 18, 3: 51, 4: 36}


In [45]:
id_col = patient_present_df['record_id']
patient_present_df = patient_present_df.drop(['record_id', 'identifier'], axis=1)

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(patient_present_df, combined_adverse_events, test_size=0.4, stratify=combined_adverse_events, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test, random_state=42)

In [47]:
y_train[y_train != 0] = 1
y_test[y_test != 0] = 1
y_val[y_val != 0] = 1

In [48]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer

imputer_categorical = SimpleImputer(strategy='most_frequent')
X_train[categorical + binary] = imputer_categorical.fit_transform(X_train[categorical + binary])
X_test[categorical + binary] = imputer_categorical.transform(X_test[categorical + binary])
X_val[categorical + binary] = imputer_categorical.transform(X_val[categorical + binary])

imputer = IterativeImputer(random_state=0)
columns = list(X_train.columns)
X_train = pd.DataFrame(data = imputer.fit_transform(X_train), columns = columns)
X_test = pd.DataFrame(data = imputer.transform(X_test), columns = columns)
X_val = pd.DataFrame(data = imputer.transform(X_val), columns = columns)

X_train

,sex,age,adhoc_pci,weight,height,clinical_presentation,diabet,hypertension,smoking,dyslipidemia,...,stent_type_2___5,stent_type_2___6,stent_type_2___7,stent_type_2___8,uncross_strategy___1,uncross_strategy___2,uncross_strategy___3,uncross_strategy___4,uncross_strategy___8,uncross_strategy___9
0,1.0,41.0,0.0,100.000000,177.000000,5.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,56.0,1.0,81.000000,172.000000,4.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,55.0,1.0,87.535524,172.093993,3.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,79.0,0.0,61.000000,160.000000,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,61.0,1.0,84.000000,179.000000,2.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,2.0,69.0,0.0,111.000000,159.000000,1.0,2.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1222,1.0,57.0,0.0,96.000000,178.000000,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1223,1.0,63.0,0.0,80.000000,162.000000,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1224,1.0,75.0,0.0,165.000000,71.000000,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)
df_for_ohe = pd.concat([X_train[categorical], X_test[categorical], X_val[categorical]], ignore_index=True)
ohe.fit(df_for_ohe)

OneHotEncoder(sparse_output=False)

In [50]:
def process_ohe(dataframe, cat_cols, encoder):
    encoded_columns = pd.DataFrame(encoder.transform(dataframe[cat_cols]))

    encoded_columns.columns = encoder.get_feature_names_out(cat_cols)

    dataframe = pd.concat([dataframe, encoded_columns], axis=1)

    dataframe.drop(cat_cols, axis=1, inplace=True)

    return dataframe

In [28]:
X_train = process_ohe(X_train, categorical, ohe)
X_test = process_ohe(X_test, categorical, ohe)
X_val = process_ohe(X_val, categorical, ohe)
X_train

,age,adhoc_pci,weight,height,diabet,hypertension,smoking,dyslipidemia,anemia,atrial_fibrilation,...,stent_number_bif_3.0,stent_number_bif_4.0,stent_technique_0.0,stent_technique_1.0,stent_direction_1.0,stent_direction_2.0,stent_direction_3.0,stent_direction_4.0,stent_direction_5.0,stent_direction_6.0
0,41.0,0.0,100.000000,177.000000,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,56.0,1.0,81.000000,172.000000,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,55.0,1.0,87.535524,172.093993,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,79.0,0.0,61.000000,160.000000,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,61.0,1.0,84.000000,179.000000,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,69.0,0.0,111.000000,159.000000,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1222,57.0,0.0,96.000000,178.000000,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1223,63.0,0.0,80.000000,162.000000,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1224,75.0,0.0,165.000000,71.000000,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [29]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

feature_selector = SelectKBest(f_classif, k=40)
X_feature_selection = feature_selector.fit_transform(X_train, y_train)
X_feature_selection.shape

strong_cols = []

feature_scores = feature_selector.scores_
features = X_train.columns
features_scores_sorted = sorted(zip(features, feature_scores), key=lambda x: x[1], reverse=True)
for col in features_scores_sorted[:30]:
    strong_cols.append(col[0])

In [30]:
X_train = X_train[strong_cols]
X_test = X_test[strong_cols]
X_val = X_val[strong_cols]
X_train

,age,anemia,ef,cerebrovascular_disease,ckd,peripheral_artery_disease,if_yes_what_type___1,creatinine,single_vessel,calcium,...,stent_diameter,minor_criteria,stent_length,adhoc_pci,previous_pci,stent_type___4,cto_bifurc,valvular_disease,side_diametr,major_lm
0,41.0,0.0,49.0,0.0,91.130343,0.0,0.0,90.0,0.0,0.0,...,4.00,1.0,11.0,0.0,1.0,0.0,0.0,0.000000,2.4,0.0
1,56.0,0.0,68.0,0.0,83.132035,0.0,0.0,89.0,1.0,0.0,...,2.75,0.0,18.0,1.0,1.0,1.0,0.0,0.000000,3.0,0.0
2,55.0,0.0,70.0,0.0,78.364633,0.0,0.0,94.0,1.0,0.0,...,2.75,2.0,33.0,1.0,0.0,1.0,0.0,0.000000,2.4,0.0
3,79.0,0.0,73.0,1.0,64.542788,0.0,0.0,96.0,0.0,1.0,...,3.50,3.0,38.0,0.0,0.0,0.0,0.0,0.000000,3.1,0.0
4,61.0,0.0,62.0,0.0,87.329657,0.0,0.0,83.0,0.0,0.0,...,4.00,2.0,28.0,1.0,0.0,0.0,0.0,0.000000,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,69.0,0.0,60.0,0.0,69.608441,0.0,0.0,97.0,1.0,0.0,...,3.50,0.0,32.0,0.0,1.0,0.0,0.0,-0.048430,2.7,0.0
1222,57.0,0.0,39.0,0.0,53.707072,0.0,0.0,127.0,0.0,0.0,...,3.50,2.0,23.0,0.0,0.0,0.0,0.0,0.005881,2.4,0.0
1223,63.0,1.0,71.0,0.0,86.111321,0.0,0.0,83.0,1.0,0.0,...,2.75,2.0,18.0,0.0,1.0,0.0,0.0,0.000000,2.2,0.0
1224,75.0,0.0,69.0,1.0,68.979615,1.0,0.0,93.0,1.0,0.0,...,3.00,0.0,18.0,0.0,1.0,0.0,0.0,0.000000,2.4,0.0


In [31]:
X_train.columns

Index(['age', 'anemia', 'ef', 'cerebrovascular_disease', 'ckd',
       'peripheral_artery_disease', 'if_yes_what_type___1', 'creatinine',
       'single_vessel', 'calcium', 'stent_type___3', 'medina_side',
       'atrial_fibrilation', 'height', 'def', 'history_of_cancer',
       'stent_type___5', 'side_stenosis', 'side_predilat',
       'previous_stroke_tia', 'stent_diameter', 'minor_criteria',
       'stent_length', 'adhoc_pci', 'previous_pci', 'stent_type___4',
       'cto_bifurc', 'valvular_disease', 'side_diametr', 'major_lm'],
      dtype='object')

In [32]:
columns_to_preserve

['valvular_disease', 'previous_stroke_tia', 'twostent_technique']

In [33]:
corr_matrix = X_train.corr().abs()

upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

threshold = 0.4

to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]

to_drop

['ckd',
 'creatinine',
 'side_stenosis',
 'side_predilat',
 'minor_criteria',
 'valvular_disease',
 'side_diametr',
 'major_lm']

In [34]:
X_train = X_train.drop(columns=to_drop)
X_test = X_test.drop(columns=to_drop)
X_val = X_val.drop(columns=to_drop)

X_train

,age,anemia,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,stent_type___3,medina_side,...,def,history_of_cancer,stent_type___5,previous_stroke_tia,stent_diameter,stent_length,adhoc_pci,previous_pci,stent_type___4,cto_bifurc
0,41.0,0.0,49.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.477395,4.00,11.0,0.0,1.0,0.0,0.0
1,56.0,0.0,68.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.313995,2.75,18.0,1.0,1.0,1.0,0.0
2,55.0,0.0,70.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.478905,2.75,33.0,1.0,0.0,1.0,0.0
3,79.0,0.0,73.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,3.50,38.0,0.0,0.0,0.0,0.0
4,61.0,0.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.448962,4.00,28.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,69.0,0.0,60.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.525775,3.50,32.0,0.0,1.0,0.0,0.0
1222,57.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.616766,3.50,23.0,0.0,0.0,0.0,0.0
1223,63.0,1.0,71.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.429977,2.75,18.0,0.0,1.0,0.0,0.0
1224,75.0,0.0,69.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.000000,3.00,18.0,0.0,1.0,0.0,0.0


In [35]:
X_train.columns

Index(['age', 'anemia', 'ef', 'cerebrovascular_disease',
       'peripheral_artery_disease', 'if_yes_what_type___1', 'single_vessel',
       'calcium', 'stent_type___3', 'medina_side', 'atrial_fibrilation',
       'height', 'def', 'history_of_cancer', 'stent_type___5',
       'previous_stroke_tia', 'stent_diameter', 'stent_length', 'adhoc_pci',
       'previous_pci', 'stent_type___4', 'cto_bifurc'],
      dtype='object')

In [36]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)

(1226, 22)
(409, 22)
(409, 22)
(1226,)


In [37]:
print(X_train.columns)

Index(['age', 'anemia', 'ef', 'cerebrovascular_disease',
       'peripheral_artery_disease', 'if_yes_what_type___1', 'single_vessel',
       'calcium', 'stent_type___3', 'medina_side', 'atrial_fibrilation',
       'height', 'def', 'history_of_cancer', 'stent_type___5',
       'previous_stroke_tia', 'stent_diameter', 'stent_length', 'adhoc_pci',
       'previous_pci', 'stent_type___4', 'cto_bifurc'],
      dtype='object')


In [38]:
X_test['height']

0      150.0
1      164.0
2      176.0
3      174.0
4      164.0
       ...  
404    172.0
405    160.0
406    177.0
407    153.0
408    168.0
Name: height, Length: 409, dtype: float64

# edge case

In [60]:
import importlib
import edge_case
importlib.reload(edge_case)
from edge_case import generate_edge_cases

edge_cases = generate_edge_cases(num_samples=150)
edge_cases


,age,height,clinical_presentation,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,medina_side,...,restenosis_reocclusion,adhoc_pci,main_predilatation,stent_diameter,stent_length,mortality,smoking,dyslipidemia,anemia,atrial_fibrilation
0,89.823707,150.220721,5,24.600405,1,0,1,1,1,0,...,1,1,1,2.722511,37.578413,1,1,1,1,1
1,93.962360,150.415046,5,19.867604,0,1,1,1,1,1,...,1,1,1,2.391960,33.328378,1,1,1,1,1
2,91.831324,159.874219,5,28.957980,1,0,0,1,1,1,...,1,1,0,2.299864,36.796772,1,1,1,1,1
3,96.168013,167.405027,5,24.534999,1,1,0,1,1,1,...,1,0,0,2.125238,28.142557,1,1,0,0,1
4,91.843633,140.667060,5,21.585955,1,1,0,1,1,1,...,1,1,1,2.316059,37.972836,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,91.503880,160.987817,5,28.844213,1,1,1,0,1,0,...,1,1,1,2.657831,29.412166,1,1,1,1,1
146,85.260353,169.702661,4,17.781154,1,1,1,1,1,1,...,1,1,0,2.523909,31.775653,1,1,1,1,0
147,94.332098,165.910680,5,16.181055,1,1,1,1,1,1,...,1,1,1,2.367857,32.869721,1,1,1,1,1
148,91.200231,160.041468,5,15.530724,1,0,0,1,1,1,...,1,1,1,2.196972,35.672523,1,1,0,1,1


In [61]:
edge_cases.columns

Index(['age', 'height', 'clinical_presentation', 'ef',
       'cerebrovascular_disease', 'peripheral_artery_disease',
       'if_yes_what_type___1', 'single_vessel', 'calcium', 'medina_side',
       'trifurcation', 'cto_bifurc', 'def', 'history_of_cancer',
       'previous_pci', 'previous_stroke_tia', 'side_diametr', 'stent_type___3',
       'stent_type___4', 'stent_type___5', 'restenosis_reocclusion',
       'adhoc_pci', 'main_predilatation', 'stent_diameter', 'stent_length',
       'mortality', 'smoking', 'dyslipidemia', 'anemia', 'atrial_fibrilation'],
      dtype='object')

In [62]:
edge_case_cols = set(edge_cases.columns)
x_train_cols = set(X_train.columns)

common_cols = edge_case_cols.intersection(x_train_cols)
only_in_edge_cases = edge_case_cols - x_train_cols
only_in_x_train = x_train_cols - edge_case_cols

print(f"COMPARISON OF COLUMNS:")
print(f"\nTotal columns in edge_cases: {len(edge_case_cols)}")
print(f"Total columns in X_train: {len(x_train_cols)}")
print(f"Number of common columns: {len(common_cols)}")

print("\n--- COMMON COLUMNS ---")
for col in sorted(common_cols):
    print(f"- {col}")

print("\n--- COLUMNS ONLY IN EDGE CASES ---")
for col in sorted(only_in_edge_cases):
    print(f"- {col}")

print("\n--- COLUMNS ONLY IN X_TRAIN ---")
for col in sorted(only_in_x_train):
    print(f"- {col}")

COMPARISON OF COLUMNS:

Total columns in edge_cases: 30
Total columns in X_train: 22
Number of common columns: 22

--- COMMON COLUMNS ---
- adhoc_pci
- age
- anemia
- atrial_fibrilation
- calcium
- cerebrovascular_disease
- cto_bifurc
- def
- ef
- height
- history_of_cancer
- if_yes_what_type___1
- medina_side
- peripheral_artery_disease
- previous_pci
- previous_stroke_tia
- single_vessel
- stent_diameter
- stent_length
- stent_type___3
- stent_type___4
- stent_type___5

--- COLUMNS ONLY IN EDGE CASES ---
- clinical_presentation
- dyslipidemia
- main_predilatation
- mortality
- restenosis_reocclusion
- side_diametr
- smoking
- trifurcation

--- COLUMNS ONLY IN X_TRAIN ---


In [56]:
edge_cases_target = edge_cases['mortality']
edge_cases_features = edge_cases.copy().drop('mortality', axis=1)

# Syntetic data

In [42]:
X_train_with_ID = X_train.copy().reset_index(drop=False)
X_train_with_ID

,index,age,anemia,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,stent_type___3,...,def,history_of_cancer,stent_type___5,previous_stroke_tia,stent_diameter,stent_length,adhoc_pci,previous_pci,stent_type___4,cto_bifurc
0,0,41.0,0.0,49.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.477395,4.00,11.0,0.0,1.0,0.0,0.0
1,1,56.0,0.0,68.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.313995,2.75,18.0,1.0,1.0,1.0,0.0
2,2,55.0,0.0,70.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.478905,2.75,33.0,1.0,0.0,1.0,0.0
3,3,79.0,0.0,73.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.000000,3.50,38.0,0.0,0.0,0.0,0.0
4,4,61.0,0.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.448962,4.00,28.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,1221,69.0,0.0,60.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.525775,3.50,32.0,0.0,1.0,0.0,0.0
1222,1222,57.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.616766,3.50,23.0,0.0,0.0,0.0,0.0
1223,1223,63.0,1.0,71.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.429977,2.75,18.0,0.0,1.0,0.0,0.0
1224,1224,75.0,0.0,69.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.000000,3.00,18.0,0.0,1.0,0.0,0.0


In [43]:
X_train_with_ID_and_target = X_train_with_ID.copy()
X_train_with_ID_and_target['target'] = y_train
X_train_with_ID_and_target

,index,age,anemia,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,stent_type___3,...,history_of_cancer,stent_type___5,previous_stroke_tia,stent_diameter,stent_length,adhoc_pci,previous_pci,stent_type___4,cto_bifurc,target
0,0,41.0,0.0,49.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.477395,4.00,11.0,0.0,1.0,0.0,0.0,0
1,1,56.0,0.0,68.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.313995,2.75,18.0,1.0,1.0,1.0,0.0,0
2,2,55.0,0.0,70.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.478905,2.75,33.0,1.0,0.0,1.0,0.0,0
3,3,79.0,0.0,73.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.000000,3.50,38.0,0.0,0.0,0.0,0.0,0
4,4,61.0,0.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.448962,4.00,28.0,1.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,1221,69.0,0.0,60.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.525775,3.50,32.0,0.0,1.0,0.0,0.0,1
1222,1222,57.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.616766,3.50,23.0,0.0,0.0,0.0,0.0,0
1223,1223,63.0,1.0,71.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.429977,2.75,18.0,0.0,1.0,0.0,0.0,0
1224,1224,75.0,0.0,69.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.000000,3.00,18.0,0.0,1.0,0.0,0.0,0


In [44]:
from synthcity.plugins import Plugins

# Print the list of available plugins
print(Plugins().list())

[KeOps] Warning : omp.h header is not in the path, disabling OpenMP. To fix this, you can set the environment
                  variable OMP_PATH to the location of the header before importing keopscore or pykeops,
                  e.g. using os.environ: import os; os.environ['OMP_PATH'] = '/path/to/omp/header'
[KeOps] Warning : Cuda libraries were not detected on the system or could not be loaded ; using cpu only mode


[2025-05-20T18:23:38.916022+0400][59467][CRITICAL] load failed: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.
[2025-05-20T18:23:38.916698+0400][59467][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'
[2025-05-20T18:23:38.916944+0400][59467][CRITICAL] module plugin_great load failed
[2025-05-20T18:23:39.049026+0400][59467][CRITICAL] module disabled: /Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/synthcity/plugins/generic/plugin_goggle.py


['aim', 'image_adsgan', 'survival_ctgan', 'rtvae', 'survival_nflow', 'arf', 'bayesian_network', 'fflows', 'marginal_distributions', 'dpgan', 'pategan', 'image_cgan', 'privbayes', 'decaf', 'ddpm', 'survival_gan', 'timevae', 'timegan', 'uniform_sampler', 'nflow', 'survae', 'dummy_sampler', 'tvae', 'ctgan', 'radialgan', 'adsgan']


# Оптимизация XGBoost на auc-roc для 10 фолдов

In [45]:
# X_train_new = pd.concat([X_train, syntetic_minority_dropped])
# X_train_new

In [46]:
# y_train_new = np.concatenate((y_train, syntetic_target))
# y_train_new

In [47]:
from sklearn.preprocessing import StandardScaler

scaller = StandardScaler()
scaller.fit(X_train)

StandardScaler()

In [48]:
import joblib

joblib.dump(scaller, "./scaler.save")

['./scaler.save']

In [49]:
X_train_folds = pd.concat([X_train, X_val])
y_train_k_fold = np.concatenate((y_train, y_val), axis=0)

In [50]:
scaled_features = scaller.transform(X_train_folds)

In [51]:
train_df = pd.DataFrame(data=scaled_features, columns=X_train_folds.columns)
train_df

,age,anemia,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,stent_type___3,medina_side,...,def,history_of_cancer,stent_type___5,previous_stroke_tia,stent_diameter,stent_length,adhoc_pci,previous_pci,stent_type___4,cto_bifurc
0,-2.317888,-0.228824,-0.659638,-0.379021,-0.288165,-0.155651,-0.926054,-0.517531,1.297194,-0.708838,...,-0.075779,-0.232745,-0.111294,0.162955,1.435755,-1.606619,-0.834094,1.163257,-0.360491,-0.286501
1,-0.791826,-0.228824,1.121135,-0.379021,-0.288165,-0.155651,1.079851,-0.517531,-0.770894,1.410760,...,-0.075779,4.296547,-0.111294,-0.730225,-0.985738,-0.768527,1.198906,1.163257,2.773993,-0.286501
2,-0.893563,-0.228824,1.308584,-0.379021,-0.288165,-0.155651,1.079851,-0.517531,-0.770894,-0.708838,...,-0.075779,-0.232745,-0.111294,0.171210,-0.985738,1.027385,1.198906,-0.859655,2.773993,-0.286501
3,1.548135,-0.228824,1.589759,2.638378,-0.288165,-0.155651,-0.926054,1.932250,-0.770894,-0.708838,...,-0.075779,-0.232745,-0.111294,-2.446583,0.467158,1.626022,-0.834094,-0.859655,-0.360491,-0.286501
4,-0.283139,-0.228824,0.558786,-0.379021,-0.288165,-0.155651,-0.926054,-0.517531,-0.770894,-0.708838,...,-0.075779,4.296547,-0.111294,0.007535,1.435755,0.428748,1.198906,-0.859655,-0.360491,-0.286501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1630,-0.690089,-0.228824,0.465061,-0.379021,-0.288165,-0.155651,1.079851,-0.517531,-0.770894,1.410760,...,-0.075779,-0.232745,-0.111294,0.080058,0.467158,-0.768527,-0.834094,1.163257,-0.360491,-0.286501
1631,1.649873,4.370167,-0.940812,-0.379021,-0.288165,-0.155651,-0.926054,1.932250,1.297194,-0.708838,...,-0.075779,-0.232745,-0.111294,1.474545,-0.501439,-1.127710,-0.834094,1.163257,-0.360491,-0.286501
1632,0.429023,-0.228824,0.652510,-0.379021,-0.288165,-0.155651,-0.926054,1.932250,-0.770894,-0.708838,...,-0.075779,4.296547,-0.111294,-0.326126,0.467158,-0.169890,-0.834094,1.163257,-0.360491,-0.286501
1633,-0.283139,-0.228824,0.090161,-0.379021,-0.288165,6.424629,1.079851,1.932250,-0.770894,1.410760,...,-0.075779,-0.232745,-0.111294,-0.335178,-0.501439,0.668203,1.198906,1.163257,-0.360491,-0.286501


# Pure XGBoost

In [52]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
import numpy as np
import pandas as pd
import pickle
import sys
import warnings
import xgboost as xgb
import json
import os

# Suppress warnings
warnings.filterwarnings("ignore")

def objective(params):
    # Convert parameters to proper format
    params['learning_rate'] = float(params['learning_rate'])
    params['max_depth'] = int(params['max_depth'])
    params['n_estimators'] = int(params['n_estimators'])
    params['min_child_weight'] = int(params['min_child_weight'])
    params['subsample'] = float(params['subsample'])
    params['colsample_bytree'] = float(params['colsample_bytree'])
    
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    roc_auc_scores = []
    f1_scores = []
    precision_scores = []
    recall_scores = []
    accuracy_scores = []

    for train_index, test_index in kf.split(X_train_folds, y_train_k_fold):
        X_train = scaller.transform(X_train_folds.iloc[train_index])
        X_test = scaller.transform(X_train_folds.iloc[test_index])
        y_train = y_train_k_fold[train_index]
        y_test = y_train_k_fold[test_index]
        
        # Initialize classifier with current parameters
        classifier = xgb.XGBClassifier(
            n_estimators=params['n_estimators'],
            max_depth=params['max_depth'],
            learning_rate=params['learning_rate'],
            min_child_weight=params['min_child_weight'],
            subsample=params['subsample'],
            colsample_bytree=params['colsample_bytree'],
            objective='binary:logistic',
            n_jobs=-1,
            random_state=42
        )
        
        # Fit the model
        classifier.fit(X_train, y_train)
        
        # Make predictions
        predictions = classifier.predict_proba(X_test)[:, 1]
        y_pred = classifier.predict(X_test)
        
        # Calculate metrics
        roc_auc = roc_auc_score(y_test, predictions)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        accuracy = accuracy_score(y_test, y_pred)
        
        roc_auc_scores.append(roc_auc)
        f1_scores.append(f1)
        precision_scores.append(precision)
        recall_scores.append(recall)
        accuracy_scores.append(accuracy)

    mean_metrics = {
        'roc_auc': np.mean(roc_auc_scores),
        'f1': np.mean(f1_scores),
        'precision': np.mean(precision_scores),
        'recall': np.mean(recall_scores),
        'accuracy': np.mean(accuracy_scores),
        'roc_auc_scores': roc_auc_scores,
        'f1_scores': f1_scores,
        'precision_scores': precision_scores,
        'recall_scores': recall_scores,
        'accuracy_scores': accuracy_scores
    }
    
    print(f"Trial completed - AUC: {mean_metrics['roc_auc']:.4f}, Params: {params}")
    
    return {'loss': -mean_metrics['roc_auc'], 'status': STATUS_OK, 'params': params, 'mean_metrics': mean_metrics}

# Define the search space for XGBoost
space = {
    'n_estimators': hp.quniform('n_estimators', 50, 500, 100),
    'max_depth': hp.choice('max_depth', [3, 4, 5, 6, 7, 8]),  # Tree depth
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.3)),  # Learning rate
    'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),  # Minimum sum of instance weight needed in a child
    'subsample': hp.uniform('subsample', 0.6, 1.0),  # Subsample ratio of the training instance
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0)  # Subsample ratio of columns when constructing each tree
}

# Run hyperparameter optimization with limited trials
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=30,
            trials=trials)

# Extract best parameters
best_trial = trials.best_trial['result']
best_params = best_trial['params']
best_metrics = best_trial['mean_metrics']

print("Hyperparameter optimization completed.")
print(f"Best parameters: {best_params}")
print(f"Best mean AUC: {best_metrics['roc_auc']:.4f}")

# Train final model with best hyperparameters on full dataset
best_model = xgb.XGBClassifier(
    n_estimators=int(best_params['n_estimators']),
    max_depth=int(best_params['max_depth']),
    learning_rate=float(best_params['learning_rate']),
    min_child_weight=int(best_params['min_child_weight']),
    subsample=float(best_params['subsample']),
    colsample_bytree=float(best_params['colsample_bytree']),
    objective='binary:logistic',
    n_jobs=-1,
    random_state=42
)

# Fit final model on the full dataset
X_train_final = scaller.transform(X_train_folds)
best_model.fit(X_train_final, y_train_k_fold)

# Save the best model
model_filename = 'best_XGBoost_model.pkl'
with open(model_filename, 'wb') as f:
    pickle.dump(best_model, f)

# Save metrics and best parameters to files
metrics_and_params = {
    'best_parameters': {
        'n_estimators': int(best_params['n_estimators']),
        'max_depth': int(best_params['max_depth']),
        'learning_rate': float(best_params['learning_rate']),
        'min_child_weight': int(best_params['min_child_weight']),
        'subsample': float(best_params['subsample']),
        'colsample_bytree': float(best_params['colsample_bytree'])
    },
    'evaluation_metrics': best_metrics
}

with open('XGBoost_metrics_and_params.json', 'w') as f:
    json.dump(metrics_and_params, f)

print("Best parameters, model, and evaluation metrics saved.")

Trial completed - AUC: 0.7020, Params: {'colsample_bytree': 0.9351394571673775, 'learning_rate': 0.014902506194388427, 'max_depth': 6, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 0.6827482895325212}
Trial completed - AUC: 0.6864, Params: {'colsample_bytree': 0.9382132654206876, 'learning_rate': 0.060797594045644714, 'max_depth': 3, 'min_child_weight': 4, 'n_estimators': 300, 'subsample': 0.9739901111585594}
Trial completed - AUC: 0.6945, Params: {'colsample_bytree': 0.9345225647939851, 'learning_rate': 0.0458040058480582, 'max_depth': 8, 'min_child_weight': 5, 'n_estimators': 100, 'subsample': 0.8149818267988931}
Trial completed - AUC: 0.7031, Params: {'colsample_bytree': 0.7755784378317518, 'learning_rate': 0.02644414129282311, 'max_depth': 3, 'min_child_weight': 4, 'n_estimators': 200, 'subsample': 0.7853139478610167}
Trial completed - AUC: 0.6816, Params: {'colsample_bytree': 0.9043106181506457, 'learning_rate': 0.027721129547826007, 'max_depth': 5, 'min_child_weight': 

In [53]:
# Train final model with best hyperparameters on full dataset
best_model = xgb.XGBClassifier(
    n_estimators=int(best_params['n_estimators']),
    max_depth=int(best_params['max_depth']),
    learning_rate=float(best_params['learning_rate']),
    min_child_weight=int(best_params['min_child_weight']),
    subsample=float(best_params['subsample']),
    colsample_bytree=float(best_params['colsample_bytree']),
    objective='binary:logistic',
    n_jobs=-1,
    random_state=42
)

# Fit final model on the full training dataset
X_train_final = scaller.transform(X_train_folds)
best_model.fit(X_train_final, y_train_k_fold)

# Save the best model
model_filename = 'pure_XGBoost.pkl'
with open(model_filename, 'wb') as f:
    pickle.dump(best_model, f)

# Evaluate the model on the test set
X_test_scaled = scaller.transform(X_test)
test_predictions = best_model.predict_proba(X_test_scaled)[:, 1]
test_pred_class = best_model.predict(X_test_scaled)

# Calculate test metrics
test_roc_auc = roc_auc_score(y_test, test_predictions)
test_f1 = f1_score(y_test, test_pred_class, zero_division=0)
test_precision = precision_score(y_test, test_pred_class, zero_division=0)
test_recall = recall_score(y_test, test_pred_class, zero_division=0)
test_accuracy = accuracy_score(y_test, test_pred_class)

test_metrics = {
    'roc_auc': test_roc_auc,
    'f1': test_f1,
    'precision': test_precision,
    'recall': test_recall,
    'accuracy': test_accuracy
}

print(f"Test metrics:")
print(f"ROC AUC: {test_roc_auc:.4f}")
print(f"F1 Score: {test_f1:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"Accuracy: {test_accuracy:.4f}")

# Save metrics and best parameters to files
metrics_and_params = {
    'test_evaluation_metrics': test_metrics,
    'best_parameters': {
        'n_estimators': int(best_params['n_estimators']),
        'max_depth': int(best_params['max_depth']),
        'learning_rate': float(best_params['learning_rate']),
        'min_child_weight': int(best_params['min_child_weight']),
        'subsample': float(best_params['subsample']),
        'colsample_bytree': float(best_params['colsample_bytree'])
    },
    'evaluation_metrics': best_metrics
}

with open('test_scores_XGBoost.json', 'w') as f:
    json.dump(metrics_and_params, f)

print("Best parameters, model, and evaluation metrics saved.")

Test metrics:
ROC AUC: 0.7922
F1 Score: 0.0000
Precision: 0.0000
Recall: 0.0000
Accuracy: 0.9242
Best parameters, model, and evaluation metrics saved.


# XGBOOST + synthetic data

In [ ]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
import numpy as np
import pandas as pd
import pickle
import sys
import warnings
import xgboost as xgb
import json
import os

# Suppress warnings
warnings.filterwarnings("ignore")

# Import synthcity components
import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import GenericDataLoader

log.add(sink=sys.stderr, level="INFO")

def objective(params):
    # Convert parameters to proper format
    params['learning_rate'] = float(params['learning_rate'])
    params['max_depth'] = int(params['max_depth'])
    params['n_estimators'] = int(params['n_estimators'])
    params['min_child_weight'] = int(params['min_child_weight'])
    params['subsample'] = float(params['subsample'])
    params['colsample_bytree'] = float(params['colsample_bytree'])
    
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    roc_auc_scores = []
    f1_scores = []
    precision_scores = []
    recall_scores = []
    accuracy_scores = []

    for train_index, test_index in kf.split(X_train_folds, y_train_k_fold):
        not_scaled_X_train_folds = X_train_folds.copy()
        scaled_X_train_folds = scaller.transform(not_scaled_X_train_folds)
        X_test = scaled_X_train_folds[test_index]
        y_test = y_train_k_fold[test_index]

        # Prepare train data with target
        train_df = not_scaled_X_train_folds.iloc[train_index].copy().reset_index(drop=True)
        train_df['target'] = y_train_k_fold[train_index]
        
        # Create GenericDataLoader as per documentation
        loader = GenericDataLoader(
            train_df,
            target_column="target",
        )
        
        # Generate synthetic data using synthcity
        syn_model = Plugins().get("arf")
        syn_model.fit(loader)
        
        # Generate synthetic samples
        synthetic_data = syn_model.generate(count=10000).dataframe()
        
        # Extract minority class samples
        minority_synthetic_data = synthetic_data[synthetic_data['target'] == 1]
        if len(minority_synthetic_data) > 600:
            minority_synthetic_data = minority_synthetic_data.sample(n=600)
        
        # Drop target column from synthetic data
        syntetic_minority_features = minority_synthetic_data.drop('target', axis=1)
        syntetic_target = minority_synthetic_data['target']

        # Combine original and synthetic data
        X_train = pd.concat([not_scaled_X_train_folds.iloc[train_index], syntetic_minority_features])
        X_train = scaller.transform(X_train)
        y_train = np.concatenate((y_train_k_fold[train_index], syntetic_target.values), axis=0)
        
        # Initialize classifier with current parameters
        classifier = xgb.XGBClassifier(
            n_estimators=params['n_estimators'],
            max_depth=params['max_depth'],
            learning_rate=params['learning_rate'],
            min_child_weight=params['min_child_weight'],
            subsample=params['subsample'],
            colsample_bytree=params['colsample_bytree'],
            objective='binary:logistic',
            n_jobs=-1,
            random_state=42
        )
        
        # Fit the model
        classifier.fit(X_train, y_train)
        
        # Make predictions
        predictions = classifier.predict_proba(X_test)[:, 1]
        y_pred = classifier.predict(X_test)
        
        # Calculate metrics
        roc_auc = roc_auc_score(y_test, predictions)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        accuracy = accuracy_score(y_test, y_pred)
        
        roc_auc_scores.append(roc_auc)
        f1_scores.append(f1)
        precision_scores.append(precision)
        recall_scores.append(recall)
        accuracy_scores.append(accuracy)

    mean_metrics = {
        'roc_auc': np.mean(roc_auc_scores),
        'f1': np.mean(f1_scores),
        'precision': np.mean(precision_scores),
        'recall': np.mean(recall_scores),
        'accuracy': np.mean(accuracy_scores),
        'roc_auc_scores': roc_auc_scores,
        'f1_scores': f1_scores,
        'precision_scores': precision_scores,
        'recall_scores': recall_scores,
        'accuracy_scores': accuracy_scores
    }
    
    print(f"Trial completed - AUC: {mean_metrics['roc_auc']:.4f}, Params: {params}")
    
    return {'loss': -mean_metrics['roc_auc'], 'status': STATUS_OK, 'params': params, 'mean_metrics': mean_metrics}

# Define the search space for XGBoost
space = {
    'n_estimators': hp.quniform('n_estimators', 100, 300, 100),
    'max_depth': hp.choice('max_depth', [4, 6, 8]),  # Tree depth
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.3)),  # Learning rate
    'min_child_weight': hp.quniform('min_child_weight', 1, 3, 1),  # Minimum sum of instance weight needed in a child
    'subsample': hp.uniform('subsample', 0.6, 1.0),  # Subsample ratio of the training instance
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0)  # Subsample ratio of columns when constructing each tree
}

# Run hyperparameter optimization with limited trials
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=15,
            trials=trials)

# Extract best parameters
best_trial = trials.best_trial['result']
best_params = best_trial['params']
best_metrics = best_trial['mean_metrics']

print("Hyperparameter optimization completed.")
print(f"Best parameters: {best_params}")
print(f"Best mean AUC: {best_metrics['roc_auc']:.4f}")

# Train final model with best hyperparameters on full dataset
best_model = xgb.XGBClassifier(
    n_estimators=int(best_params['n_estimators']),
    max_depth=int(best_params['max_depth']),
    learning_rate=float(best_params['learning_rate']),
    min_child_weight=int(best_params['min_child_weight']),
    subsample=float(best_params['subsample']),
    colsample_bytree=float(best_params['colsample_bytree']),
    objective='binary:logistic',
    n_jobs=-1,
    random_state=42
)

# Generate synthetic data for final model using full training set
train_full = X_train_folds.copy().reset_index(drop=True)
train_full['target'] = y_train_k_fold

# Create loader for full dataset
full_loader = GenericDataLoader(
    train_full,
    target_column="target"
)

# Generate synthetic data for the full dataset
syn_model = Plugins().get("arf")
syn_model.fit(full_loader)
synthetic_data = syn_model.generate(count=10000).dataframe()

# Extract minority samples
minority_synthetic_data = synthetic_data[synthetic_data['target'] == 1]
if len(minority_synthetic_data) > 600:
    minority_synthetic_data = minority_synthetic_data.sample(n=600)
syntetic_minority_features = minority_synthetic_data.drop('target', axis=1)
syntetic_target = minority_synthetic_data['target']

# Combine original and synthetic data for final training
X_train_final = pd.concat([X_train_folds, syntetic_minority_features])
X_train_final = scaller.transform(X_train_final)
y_train_final = np.concatenate((y_train_k_fold, syntetic_target.values), axis=0)

# Fit final model
best_model.fit(X_train_final, y_train_final)

# Save the best model
model_filename = 'XGBoost_arf.pkl'
with open(model_filename, 'wb') as f:
    pickle.dump(best_model, f)

# Evaluate the model on the test set
X_test_scaled = scaller.transform(X_test)
test_predictions = best_model.predict_proba(X_test_scaled)[:, 1]
test_pred_class = best_model.predict(X_test_scaled)

# Calculate test metrics
test_roc_auc = roc_auc_score(y_test, test_predictions)
test_f1 = f1_score(y_test, test_pred_class, zero_division=0)
test_precision = precision_score(y_test, test_pred_class, zero_division=0)
test_recall = recall_score(y_test, test_pred_class, zero_division=0)
test_accuracy = accuracy_score(y_test, test_pred_class)

test_metrics = {
    'roc_auc': test_roc_auc,
    'f1': test_f1,
    'precision': test_precision,
    'recall': test_recall,
    'accuracy': test_accuracy
}

print(f"Test metrics:")
print(f"ROC AUC: {test_roc_auc:.4f}")
print(f"F1 Score: {test_f1:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"Accuracy: {test_accuracy:.4f}")

# Save metrics and best parameters to files
metrics_and_params = {
    'test_evaluation_metrics': test_metrics,
    'best_parameters': {
        'n_estimators': int(best_params['n_estimators']),
        'max_depth': int(best_params['max_depth']),
        'learning_rate': float(best_params['learning_rate']),
        'min_child_weight': int(best_params['min_child_weight']),
        'subsample': float(best_params['subsample']),
        'colsample_bytree': float(best_params['colsample_bytree'])
    },
    'evaluation_metrics': best_metrics
}

with open('XGBoost_arf.json', 'w') as f:
    json.dump(metrics_and_params, f)

print("Best parameters, model, and evaluation metrics saved.")

# XGBOOST + ARF + EDGE CASES

In [63]:
edge_cases_features_names = [feature for feature in edge_cases.columns if feature in X_train.columns]
edge_cases_features = pd.DataFrame(edge_cases, columns=edge_cases_features_names)
edge_cases_features

,age,height,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,medina_side,cto_bifurc,...,previous_pci,previous_stroke_tia,stent_type___3,stent_type___4,stent_type___5,adhoc_pci,stent_diameter,stent_length,anemia,atrial_fibrilation
0,89.823707,150.220721,24.600405,1,0,1,1,1,0,1,...,1,0,0,0,1,1,2.722511,37.578413,1,1
1,93.962360,150.415046,19.867604,0,1,1,1,1,1,1,...,1,0,1,0,0,1,2.391960,33.328378,1,1
2,91.831324,159.874219,28.957980,1,0,0,1,1,1,1,...,0,0,1,1,1,1,2.299864,36.796772,1,1
3,96.168013,167.405027,24.534999,1,1,0,1,1,1,0,...,0,1,0,1,1,0,2.125238,28.142557,0,1
4,91.843633,140.667060,21.585955,1,1,0,1,1,1,0,...,1,0,0,1,1,1,2.316059,37.972836,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,91.503880,160.987817,28.844213,1,1,1,0,1,0,1,...,0,0,1,0,1,1,2.657831,29.412166,1,1
146,85.260353,169.702661,17.781154,1,1,1,1,1,1,1,...,1,0,0,1,0,1,2.523909,31.775653,1,0
147,94.332098,165.910680,16.181055,1,1,1,1,1,1,1,...,1,1,1,1,1,1,2.367857,32.869721,1,1
148,91.200231,160.041468,15.530724,1,0,0,1,1,1,1,...,1,1,1,0,0,1,2.196972,35.672523,1,1


In [64]:
X_train_folds = pd.concat([X_train, X_val, edge_cases_features])
y_train_k_fold = np.concatenate((y_train, y_val, edge_cases_target), axis=0)

In [ ]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
import numpy as np
import pandas as pd
import pickle
import sys
import warnings
import xgboost as xgb
import json
import os

# Suppress warnings
warnings.filterwarnings("ignore")

# Import synthcity components
import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import GenericDataLoader

log.add(sink=sys.stderr, level="INFO")

def objective(params):
    # Convert parameters to proper format
    params['learning_rate'] = float(params['learning_rate'])
    params['max_depth'] = int(params['max_depth'])
    params['n_estimators'] = int(params['n_estimators'])
    params['min_child_weight'] = int(params['min_child_weight'])
    params['subsample'] = float(params['subsample'])
    params['colsample_bytree'] = float(params['colsample_bytree'])
    
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    roc_auc_scores = []
    f1_scores = []
    precision_scores = []
    recall_scores = []
    accuracy_scores = []

    for train_index, test_index in kf.split(X_train_folds, y_train_k_fold):
        not_scaled_X_train_folds = X_train_folds.copy()
        scaled_X_train_folds = scaller.transform(not_scaled_X_train_folds)
        X_test = scaled_X_train_folds[test_index]
        y_test = y_train_k_fold[test_index]

        # Prepare train data with target
        train_df = not_scaled_X_train_folds.iloc[train_index].copy().reset_index(drop=True)
        train_df['target'] = y_train_k_fold[train_index]
        
        # Create GenericDataLoader as per documentation
        loader = GenericDataLoader(
            train_df,
            target_column="target",
        )
        
        # Generate synthetic data using synthcity
        syn_model = Plugins().get("arf")
        syn_model.fit(loader)
        
        # Generate synthetic samples
        synthetic_data = syn_model.generate(count=10000).dataframe()
        
        # Extract minority class samples
        minority_synthetic_data = synthetic_data[synthetic_data['target'] == 1]
        if len(minority_synthetic_data) > 600:
            minority_synthetic_data = minority_synthetic_data.sample(n=600)
        
        # Drop target column from synthetic data
        syntetic_minority_features = minority_synthetic_data.drop('target', axis=1)
        syntetic_target = minority_synthetic_data['target']

        # Combine original and synthetic data
        X_train = pd.concat([not_scaled_X_train_folds.iloc[train_index], syntetic_minority_features])
        X_train = scaller.transform(X_train)
        y_train = np.concatenate((y_train_k_fold[train_index], syntetic_target.values), axis=0)
        
        # Initialize classifier with current parameters
        classifier = xgb.XGBClassifier(
            n_estimators=params['n_estimators'],
            max_depth=params['max_depth'],
            learning_rate=params['learning_rate'],
            min_child_weight=params['min_child_weight'],
            subsample=params['subsample'],
            colsample_bytree=params['colsample_bytree'],
            objective='binary:logistic',
            n_jobs=-1,
            random_state=42
        )
        
        # Fit the model
        classifier.fit(X_train, y_train)
        
        # Make predictions
        predictions = classifier.predict_proba(X_test)[:, 1]
        y_pred = classifier.predict(X_test)
        
        # Calculate metrics
        roc_auc = roc_auc_score(y_test, predictions)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        accuracy = accuracy_score(y_test, y_pred)
        
        roc_auc_scores.append(roc_auc)
        f1_scores.append(f1)
        precision_scores.append(precision)
        recall_scores.append(recall)
        accuracy_scores.append(accuracy)

    mean_metrics = {
        'roc_auc': np.mean(roc_auc_scores),
        'f1': np.mean(f1_scores),
        'precision': np.mean(precision_scores),
        'recall': np.mean(recall_scores),
        'accuracy': np.mean(accuracy_scores),
        'roc_auc_scores': roc_auc_scores,
        'f1_scores': f1_scores,
        'precision_scores': precision_scores,
        'recall_scores': recall_scores,
        'accuracy_scores': accuracy_scores
    }
    
    print(f"Trial completed - AUC: {mean_metrics['roc_auc']:.4f}, Params: {params}")
    
    return {'loss': -mean_metrics['roc_auc'], 'status': STATUS_OK, 'params': params, 'mean_metrics': mean_metrics}

# Define the search space for XGBoost
space = {
    'n_estimators': hp.quniform('n_estimators', 100, 300, 100),
    'max_depth': hp.choice('max_depth', [4, 6, 8]),  # Tree depth
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.3)),  # Learning rate
    'min_child_weight': hp.quniform('min_child_weight', 1, 3, 1),  # Minimum sum of instance weight needed in a child
    'subsample': hp.uniform('subsample', 0.6, 1.0),  # Subsample ratio of the training instance
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0)  # Subsample ratio of columns when constructing each tree
}

# Run hyperparameter optimization with limited trials
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=15,
            trials=trials)

# Extract best parameters
best_trial = trials.best_trial['result']
best_params = best_trial['params']
best_metrics = best_trial['mean_metrics']

print("Hyperparameter optimization completed.")
print(f"Best parameters: {best_params}")
print(f"Best mean AUC: {best_metrics['roc_auc']:.4f}")

# Train final model with best hyperparameters on full dataset
best_model = xgb.XGBClassifier(
    n_estimators=int(best_params['n_estimators']),
    max_depth=int(best_params['max_depth']),
    learning_rate=float(best_params['learning_rate']),
    min_child_weight=int(best_params['min_child_weight']),
    subsample=float(best_params['subsample']),
    colsample_bytree=float(best_params['colsample_bytree']),
    objective='binary:logistic',
    n_jobs=-1,
    random_state=42
)

# Generate synthetic data for final model using full training set
train_full = X_train_folds.copy().reset_index(drop=True)
train_full['target'] = y_train_k_fold

# Create loader for full dataset
full_loader = GenericDataLoader(
    train_full,
    target_column="target"
)

# Generate synthetic data for the full dataset
syn_model = Plugins().get("arf")
syn_model.fit(full_loader)
synthetic_data = syn_model.generate(count=10000).dataframe()

# Extract minority samples
minority_synthetic_data = synthetic_data[synthetic_data['target'] == 1]
if len(minority_synthetic_data) > 600:
    minority_synthetic_data = minority_synthetic_data.sample(n=600)
syntetic_minority_features = minority_synthetic_data.drop('target', axis=1)
syntetic_target = minority_synthetic_data['target']

# Combine original and synthetic data for final training
X_train_final = pd.concat([X_train_folds, syntetic_minority_features])
X_train_final = scaller.fit_transform(X_train_final)
y_train_final = np.concatenate((y_train_k_fold, syntetic_target.values), axis=0)

# Fit final model
best_model.fit(X_train_final, y_train_final)

# Save the best model
model_filename = 'XGBoost_arf_edge_cases.pkl'
with open(model_filename, 'wb') as f:
    pickle.dump(best_model, f)

# Evaluate the model on the test set
X_test_scaled = scaller.transform(X_test)
test_predictions = best_model.predict_proba(X_test_scaled)[:, 1]
test_pred_class = best_model.predict(X_test_scaled)

# Calculate test metrics
test_roc_auc = roc_auc_score(y_test, test_predictions)
test_f1 = f1_score(y_test, test_pred_class, zero_division=0)
test_precision = precision_score(y_test, test_pred_class, zero_division=0)
test_recall = recall_score(y_test, test_pred_class, zero_division=0)
test_accuracy = accuracy_score(y_test, test_pred_class)

test_metrics = {
    'roc_auc': test_roc_auc,
    'f1': test_f1,
    'precision': test_precision,
    'recall': test_recall,
    'accuracy': test_accuracy
}

print(f"Test metrics:")
print(f"ROC AUC: {test_roc_auc:.4f}")
print(f"F1 Score: {test_f1:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"Accuracy: {test_accuracy:.4f}")

# Save metrics and best parameters to files
metrics_and_params = {
    'test_evaluation_metrics': test_metrics,
    'best_parameters': {
        'n_estimators': int(best_params['n_estimators']),
        'max_depth': int(best_params['max_depth']),
        'learning_rate': float(best_params['learning_rate']),
        'min_child_weight': int(best_params['min_child_weight']),
        'subsample': float(best_params['subsample']),
        'colsample_bytree': float(best_params['colsample_bytree'])
    },
    'evaluation_metrics': best_metrics
}

with open('XGBoost_arf_edge_cases.json', 'w') as f:
    json.dump(metrics_and_params, f)

print("Best parameters, model, and evaluation metrics saved.")

# XGBoost + edge cases

In [66]:
edge_cases_features_names = [feature for feature in edge_cases.columns if feature in X_train.columns]
edge_cases_features = pd.DataFrame(edge_cases, columns=edge_cases_features_names)
edge_cases_features

,age,height,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,medina_side,cto_bifurc,...,previous_pci,previous_stroke_tia,stent_type___3,stent_type___4,stent_type___5,adhoc_pci,stent_diameter,stent_length,anemia,atrial_fibrilation
0,89.823707,150.220721,24.600405,1,0,1,1,1,0,1,...,1,0,0,0,1,1,2.722511,37.578413,1,1
1,93.962360,150.415046,19.867604,0,1,1,1,1,1,1,...,1,0,1,0,0,1,2.391960,33.328378,1,1
2,91.831324,159.874219,28.957980,1,0,0,1,1,1,1,...,0,0,1,1,1,1,2.299864,36.796772,1,1
3,96.168013,167.405027,24.534999,1,1,0,1,1,1,0,...,0,1,0,1,1,0,2.125238,28.142557,0,1
4,91.843633,140.667060,21.585955,1,1,0,1,1,1,0,...,1,0,0,1,1,1,2.316059,37.972836,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,91.503880,160.987817,28.844213,1,1,1,0,1,0,1,...,0,0,1,0,1,1,2.657831,29.412166,1,1
146,85.260353,169.702661,17.781154,1,1,1,1,1,1,1,...,1,0,0,1,0,1,2.523909,31.775653,1,0
147,94.332098,165.910680,16.181055,1,1,1,1,1,1,1,...,1,1,1,1,1,1,2.367857,32.869721,1,1
148,91.200231,160.041468,15.530724,1,0,0,1,1,1,1,...,1,1,1,0,0,1,2.196972,35.672523,1,1


In [67]:
X_train_folds = pd.concat([X_train, X_val, edge_cases_features])
y_train_k_fold = np.concatenate((y_train, y_val, edge_cases_target), axis=0)

In [68]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
import numpy as np
import pandas as pd
import pickle
import sys
import warnings
import xgboost as xgb
import json
import os

# Suppress warnings
warnings.filterwarnings("ignore")

def objective(params):
    # Convert parameters to proper format
    params['learning_rate'] = float(params['learning_rate'])
    params['max_depth'] = int(params['max_depth'])
    params['n_estimators'] = int(params['n_estimators'])
    params['min_child_weight'] = int(params['min_child_weight'])
    params['subsample'] = float(params['subsample'])
    params['colsample_bytree'] = float(params['colsample_bytree'])
    
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    roc_auc_scores = []
    f1_scores = []
    precision_scores = []
    recall_scores = []
    accuracy_scores = []

    for train_index, test_index in kf.split(X_train_folds, y_train_k_fold):
        X_train = scaller.transform(X_train_folds.iloc[train_index])
        X_test = scaller.transform(X_train_folds.iloc[test_index])
        y_train = y_train_k_fold[train_index]
        y_test = y_train_k_fold[test_index]
        
        # Initialize classifier with current parameters
        classifier = xgb.XGBClassifier(
            n_estimators=params['n_estimators'],
            max_depth=params['max_depth'],
            learning_rate=params['learning_rate'],
            min_child_weight=params['min_child_weight'],
            subsample=params['subsample'],
            colsample_bytree=params['colsample_bytree'],
            objective='binary:logistic',
            n_jobs=-1,
            random_state=42
        )
        
        # Fit the model
        classifier.fit(X_train, y_train)
        
        # Make predictions
        predictions = classifier.predict_proba(X_test)[:, 1]
        y_pred = classifier.predict(X_test)
        
        # Calculate metrics
        roc_auc = roc_auc_score(y_test, predictions)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        accuracy = accuracy_score(y_test, y_pred)
        
        roc_auc_scores.append(roc_auc)
        f1_scores.append(f1)
        precision_scores.append(precision)
        recall_scores.append(recall)
        accuracy_scores.append(accuracy)

    mean_metrics = {
        'roc_auc': np.mean(roc_auc_scores),
        'f1': np.mean(f1_scores),
        'precision': np.mean(precision_scores),
        'recall': np.mean(recall_scores),
        'accuracy': np.mean(accuracy_scores),
        'roc_auc_scores': roc_auc_scores,
        'f1_scores': f1_scores,
        'precision_scores': precision_scores,
        'recall_scores': recall_scores,
        'accuracy_scores': accuracy_scores
    }
    
    print(f"Trial completed - AUC: {mean_metrics['roc_auc']:.4f}, Params: {params}")
    
    return {'loss': -mean_metrics['roc_auc'], 'status': STATUS_OK, 'params': params, 'mean_metrics': mean_metrics}

# Define the search space for XGBoost
space = {
    'n_estimators': hp.quniform('n_estimators', 50, 500, 100),
    'max_depth': hp.choice('max_depth', [3, 4, 5, 6, 7, 8]),  # Tree depth
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.3)),  # Learning rate
    'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),  # Minimum sum of instance weight needed in a child
    'subsample': hp.uniform('subsample', 0.6, 1.0),  # Subsample ratio of the training instance
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0)  # Subsample ratio of columns when constructing each tree
}

# Run hyperparameter optimization with limited trials
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=30,
            trials=trials)

# Extract best parameters
best_trial = trials.best_trial['result']
best_params = best_trial['params']
best_metrics = best_trial['mean_metrics']

print("Hyperparameter optimization completed.")
print(f"Best parameters: {best_params}")
print(f"Best mean AUC: {best_metrics['roc_auc']:.4f}")

# Train final model with best hyperparameters on full dataset
best_model = xgb.XGBClassifier(
    n_estimators=int(best_params['n_estimators']),
    max_depth=int(best_params['max_depth']),
    learning_rate=float(best_params['learning_rate']),
    min_child_weight=int(best_params['min_child_weight']),
    subsample=float(best_params['subsample']),
    colsample_bytree=float(best_params['colsample_bytree']),
    objective='binary:logistic',
    n_jobs=-1,
    random_state=42
)

# Fit final model on the full dataset
X_train_final = scaller.transform(X_train_folds)
best_model.fit(X_train_final, y_train_k_fold)

# Save the best model
model_filename = 'XGBoost_edge_cases.pkl'
with open(model_filename, 'wb') as f:
    pickle.dump(best_model, f)

# Save metrics and best parameters to files
metrics_and_params = {
    'best_parameters': {
        'n_estimators': int(best_params['n_estimators']),
        'max_depth': int(best_params['max_depth']),
        'learning_rate': float(best_params['learning_rate']),
        'min_child_weight': int(best_params['min_child_weight']),
        'subsample': float(best_params['subsample']),
        'colsample_bytree': float(best_params['colsample_bytree'])
    },
    'evaluation_metrics': best_metrics
}

with open('XGBoost_edge_cases.json', 'w') as f:
    json.dump(metrics_and_params, f)

print("Best parameters, model, and evaluation metrics saved.")

Trial completed - AUC: 0.8076, Params: {'colsample_bytree': 0.6878479595052424, 'learning_rate': 0.18980154379317812, 'max_depth': 7, 'min_child_weight': 2, 'n_estimators': 400, 'subsample': 0.8820243278025484}
Trial completed - AUC: 0.8418, Params: {'colsample_bytree': 0.9774100878071758, 'learning_rate': 0.019806261963009292, 'max_depth': 8, 'min_child_weight': 4, 'n_estimators': 400, 'subsample': 0.9050367972008729}
Trial completed - AUC: 0.8491, Params: {'colsample_bytree': 0.7220631419231199, 'learning_rate': 0.012833813167447073, 'max_depth': 8, 'min_child_weight': 2, 'n_estimators': 100, 'subsample': 0.867397266953601}
Trial completed - AUC: 0.8383, Params: {'colsample_bytree': 0.8934080723690822, 'learning_rate': 0.0337602679608377, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 500, 'subsample': 0.8936527990035854}
Trial completed - AUC: 0.8160, Params: {'colsample_bytree': 0.9260261829636446, 'learning_rate': 0.09427181042808193, 'max_depth': 6, 'min_child_weight': 1,

In [69]:
# Train final model with best hyperparameters on full dataset
best_model = xgb.XGBClassifier(
    n_estimators=int(best_params['n_estimators']),
    max_depth=int(best_params['max_depth']),
    learning_rate=float(best_params['learning_rate']),
    min_child_weight=int(best_params['min_child_weight']),
    subsample=float(best_params['subsample']),
    colsample_bytree=float(best_params['colsample_bytree']),
    objective='binary:logistic',
    n_jobs=-1,
    random_state=42
)

# Fit final model on the full training dataset
X_train_final = scaller.transform(X_train_folds)
best_model.fit(X_train_final, y_train_k_fold)

# Save the best model
model_filename = 'XGBoost_edge_cases.pkl'
with open(model_filename, 'wb') as f:
    pickle.dump(best_model, f)

# Evaluate the model on the test set
X_test_scaled = scaller.transform(X_test)
test_predictions = best_model.predict_proba(X_test_scaled)[:, 1]
test_pred_class = best_model.predict(X_test_scaled)

# Calculate test metrics
test_roc_auc = roc_auc_score(y_test, test_predictions)
test_f1 = f1_score(y_test, test_pred_class, zero_division=0)
test_precision = precision_score(y_test, test_pred_class, zero_division=0)
test_recall = recall_score(y_test, test_pred_class, zero_division=0)
test_accuracy = accuracy_score(y_test, test_pred_class)

test_metrics = {
    'roc_auc': test_roc_auc,
    'f1': test_f1,
    'precision': test_precision,
    'recall': test_recall,
    'accuracy': test_accuracy
}

print(f"Test metrics:")
print(f"ROC AUC: {test_roc_auc:.4f}")
print(f"F1 Score: {test_f1:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"Accuracy: {test_accuracy:.4f}")

# Save metrics and best parameters to files
metrics_and_params = {
    'test_evaluation_metrics': test_metrics,
    'best_parameters': {
        'n_estimators': int(best_params['n_estimators']),
        'max_depth': int(best_params['max_depth']),
        'learning_rate': float(best_params['learning_rate']),
        'min_child_weight': int(best_params['min_child_weight']),
        'subsample': float(best_params['subsample']),
        'colsample_bytree': float(best_params['colsample_bytree'])
    },
    'evaluation_metrics': best_metrics
}

with open('test_scores_XGBoost_edge_cases.json', 'w') as f:
    json.dump(metrics_and_params, f)

print("Best parameters, model, and evaluation metrics saved.")

Test metrics:
ROC AUC: 0.7931
F1 Score: 0.0000
Precision: 0.0000
Recall: 0.0000
Accuracy: 0.9242
Best parameters, model, and evaluation metrics saved.
